In [418]:
# https://preppindata.blogspot.com/2021/03/2021-week-12-maldives-tourism.html

import pandas as pd
import numpy as np


### Input the Data

In [419]:
df_SeatList = pd.read_excel(r'C:\Users\StanleyChan\SynologyDrive\Work Backup\my document\python\Preppin Data\data\PD 2021 Wk 14 Input.xlsx', sheet_name='SeatList')
df_Passenger = pd.read_excel(r'C:\Users\StanleyChan\SynologyDrive\Work Backup\my document\python\Preppin Data\data\PD 2021 Wk 14 Input.xlsx', sheet_name='Passenger List')
df_Flights = pd.read_excel(r'C:\Users\StanleyChan\SynologyDrive\Work Backup\my document\python\Preppin Data\data\PD 2021 Wk 14 Input.xlsx', sheet_name='FlightDetails')
df_Planes = pd.read_excel(r'C:\Users\StanleyChan\SynologyDrive\Work Backup\my document\python\Preppin Data\data\PD 2021 Wk 14 Input.xlsx', sheet_name = 'PlaneDetails')

df_SeatList.head(5)

,Row,A,B,C,D,E,F
0,1,1,2,3,4,5,6
1,2,7,8,9,10,11,12
2,3,13,14,15,16,17,18
3,4,19,20,21,22,23,24
4,5,25,26,27,28,29,30


### Assign a label for where each seat is located
They are assigned as follows:
- A & F - Window Seats
- B & E - Middle Seats
- C & D - Aisle Seats 

In [420]:
column_mapping = {
    'A': 'Window Seats',
    'B': 'Middle Seats',
    'C': 'Aisle Seats',
    'D': 'Aisle Seats',
    'E': 'Middle Seats',
    'F': 'Window Seats'
}

# Rename columns using the mapping
df_SeatList = df_SeatList.rename(columns=column_mapping)


### Combine the Seat List and Passenger List tables.

In [421]:
# reshape the table first
df_SeatList = df_SeatList.melt(id_vars='Row', var_name='Seat Position', value_name='passenger_number')
df_SeatList

,Row,Seat Position,passenger_number
0,1,Window Seats,1
1,2,Window Seats,7
2,3,Window Seats,13
3,4,Window Seats,19
4,5,Window Seats,25
...,...,...,...
115,16,Window Seats,96
116,17,Window Seats,102
117,18,Window Seats,108
118,19,Window Seats,114


In [422]:
df_Passenger = df_Passenger[['first_name', 'last_name', 'passenger_number', 'flight_number','purchase_amount']]
df_Passenger

,first_name,last_name,passenger_number,flight_number,purchase_amount
0,Jerrylee,Rein,1,1,48.29
1,Forester,Iashvili,2,1,0.00
2,Shaun,Sherwill,3,1,0.00
3,Werner,Basile,4,1,58.21
4,Kerwinn,Skillen,5,1,41.96
...,...,...,...,...,...
995,Skye,McLaverty,106,10,10.46
996,Margaux,Rymour,107,10,0.00
997,Corny,Vaszoly,108,10,44.60
998,Vittorio,Rushbrook,109,10,0.00


In [423]:
df_Passenger_w_seat = df_Passenger.merge(df_SeatList, on = 'passenger_number', how='left')
df_Passenger_w_seat

,first_name,last_name,passenger_number,flight_number,purchase_amount,Row,Seat Position
0,Jerrylee,Rein,1,1,48.29,1,Window Seats
1,Forester,Iashvili,2,1,0.00,1,Middle Seats
2,Shaun,Sherwill,3,1,0.00,1,Aisle Seats
3,Werner,Basile,4,1,58.21,1,Aisle Seats
4,Kerwinn,Skillen,5,1,41.96,1,Middle Seats
...,...,...,...,...,...,...,...
995,Skye,McLaverty,106,10,10.46,18,Aisle Seats
996,Margaux,Rymour,107,10,0.00,18,Middle Seats
997,Corny,Vaszoly,108,10,44.60,18,Window Seats
998,Vittorio,Rushbrook,109,10,0.00,19,Window Seats


### Parse the Flight Details so that they are in separate fields

In [424]:
df_Flights

,[FlightID|DepAir|ArrAir|DepDate|DepTime]
0,[1|LHR|SEA|2020-10-08|14:53:00]
1,[2|MTY|JFK|2020-12-03|06:51:00]
2,[3|SEA|BOS|2020-11-21|20:45:00]
3,[4|LHR|BOS|2020-10-31|21:01:00]
4,[5|MTY|CAI|2020-12-07|09:33:00]
5,[6|JFK|LHR|2020-11-10|05:05:00]
6,[7|TPE|LHR|2020-12-01|14:22:00]
7,[8|BOS|SEA|2020-12-26|16:45:00]
8,[9|JFK|LHR|2020-10-23|19:06:00]
9,[10|CAI|LHR|2020-12-22|10:54:00]


In [425]:
df_Flights['[FlightID|DepAir|ArrAir|DepDate|DepTime]'] = df_Flights['[FlightID|DepAir|ArrAir|DepDate|DepTime]'].str.replace('[','')
df_Flights['[FlightID|DepAir|ArrAir|DepDate|DepTime]'] = df_Flights['[FlightID|DepAir|ArrAir|DepDate|DepTime]'].str.replace(']','')
df_Flights[['FlightID', 'DepAir','ArrAir','DepDate','DepTime']] = df_Flights['[FlightID|DepAir|ArrAir|DepDate|DepTime]'].str.split('|', expand=True)
df_Flights.drop('[FlightID|DepAir|ArrAir|DepDate|DepTime]', axis=1,inplace=True)

df_Flights

,FlightID,DepAir,ArrAir,DepDate,DepTime
0,1,LHR,SEA,2020-10-08,14:53:00
1,2,MTY,JFK,2020-12-03,06:51:00
2,3,SEA,BOS,2020-11-21,20:45:00
3,4,LHR,BOS,2020-10-31,21:01:00
4,5,MTY,CAI,2020-12-07,09:33:00
5,6,JFK,LHR,2020-11-10,05:05:00
6,7,TPE,LHR,2020-12-01,14:22:00
7,8,BOS,SEA,2020-12-26,16:45:00
8,9,JFK,LHR,2020-10-23,19:06:00
9,10,CAI,LHR,2020-12-22,10:54:00


### Calculate the time of day for each flight. (Hint)
They are assigned as follows: 
- Morning - Before 12:00 
- Afternoon - Between 12:00 - 18:00
- Evening - After 18:00

In [426]:
# Convert 'DepTime' column to datetime format
df_Flights['DepTime'] = pd.to_datetime(df_Flights['DepTime'], format='%H:%M:%S')

# Reclassify based on time categories
df_Flights['Depart Time of Day'] = pd.cut(df_Flights['DepTime'].dt.hour,
                            bins=[0, 12, 18, 24],
                            labels=['Morning', 'Afternoon', 'Evening'],
                            include_lowest=True)


### Join the Flight Details & Plane Details to the Passenger & Seat tables.

In [427]:
# merge flight with plane details
df_Planes['FlightNo.'] = df_Planes['FlightNo.'].astype(str)
df_Flights_Planes = df_Flights.merge(df_Planes, left_on='FlightID',right_on='FlightNo.', how='left')
df_Flights_Planes

,FlightID,DepAir,ArrAir,DepDate,DepTime,Depart Time of Day,FlightNo.,Business Class
0,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,1-5
1,2,MTY,JFK,2020-12-03,1900-01-01 06:51:00,Morning,2,1-8
2,3,SEA,BOS,2020-11-21,1900-01-01 20:45:00,Evening,3,1-10
3,4,LHR,BOS,2020-10-31,1900-01-01 21:01:00,Evening,4,1-5
4,5,MTY,CAI,2020-12-07,1900-01-01 09:33:00,Morning,5,1-5
5,6,JFK,LHR,2020-11-10,1900-01-01 05:05:00,Morning,6,1-5
6,7,TPE,LHR,2020-12-01,1900-01-01 14:22:00,Afternoon,7,1-2
7,8,BOS,SEA,2020-12-26,1900-01-01 16:45:00,Afternoon,8,1-3
8,9,JFK,LHR,2020-10-23,1900-01-01 19:06:00,Evening,9,1-5
9,10,CAI,LHR,2020-12-22,1900-01-01 10:54:00,Morning,10,1-5


In [428]:
df_Passenger_w_seat['flight_number'] = df_Passenger_w_seat['flight_number'].astype(str)
df_all = df_Passenger_w_seat.merge(df_Flights_Planes, left_on='flight_number', right_on='FlightNo.', how='left')
df_all

,first_name,last_name,passenger_number,flight_number,purchase_amount,Row,Seat Position,FlightID,DepAir,ArrAir,DepDate,DepTime,Depart Time of Day,FlightNo.,Business Class
0,Jerrylee,Rein,1,1,48.29,1,Window Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,1-5
1,Forester,Iashvili,2,1,0.00,1,Middle Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,1-5
2,Shaun,Sherwill,3,1,0.00,1,Aisle Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,1-5
3,Werner,Basile,4,1,58.21,1,Aisle Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,1-5
4,Kerwinn,Skillen,5,1,41.96,1,Middle Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,1-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Skye,McLaverty,106,10,10.46,18,Aisle Seats,10,CAI,LHR,2020-12-22,1900-01-01 10:54:00,Morning,10,1-5
996,Margaux,Rymour,107,10,0.00,18,Middle Seats,10,CAI,LHR,2020-12-22,1900-01-01 10:54:00,Morning,10,1-5
997,Corny,Vaszoly,108,10,44.60,18,Window Seats,10,CAI,LHR,2020-12-22,1900-01-01 10:54:00,Morning,10,1-5
998,Vittorio,Rushbrook,109,10,0.00,19,Window Seats,10,CAI,LHR,2020-12-22,1900-01-01 10:54:00,Morning,10,1-5


### Identify what rows are Business or Economy Class for each flight

In [429]:
df_all[['Bclass_start','Bclass_end']] = df_all['Business Class'].str.split('-',expand=True)

df_all['Bclass_end'] = df_all['Bclass_end'].astype(int)
df_all['Business Class'] = np.where(df_all['Row'] <= df_all['Bclass_end'],'Busniess','Economy')
df_all


,first_name,last_name,passenger_number,flight_number,purchase_amount,Row,Seat Position,FlightID,DepAir,ArrAir,DepDate,DepTime,Depart Time of Day,FlightNo.,Business Class,Bclass_start,Bclass_end
0,Jerrylee,Rein,1,1,48.29,1,Window Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,Busniess,1,5
1,Forester,Iashvili,2,1,0.00,1,Middle Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,Busniess,1,5
2,Shaun,Sherwill,3,1,0.00,1,Aisle Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,Busniess,1,5
3,Werner,Basile,4,1,58.21,1,Aisle Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,Busniess,1,5
4,Kerwinn,Skillen,5,1,41.96,1,Middle Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,Busniess,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Skye,McLaverty,106,10,10.46,18,Aisle Seats,10,CAI,LHR,2020-12-22,1900-01-01 10:54:00,Morning,10,Economy,1,5
996,Margaux,Rymour,107,10,0.00,18,Middle Seats,10,CAI,LHR,2020-12-22,1900-01-01 10:54:00,Morning,10,Economy,1,5
997,Corny,Vaszoly,108,10,44.60,18,Window Seats,10,CAI,LHR,2020-12-22,1900-01-01 10:54:00,Morning,10,Economy,1,5
998,Vittorio,Rushbrook,109,10,0.00,19,Window Seats,10,CAI,LHR,2020-12-22,1900-01-01 10:54:00,Morning,10,Economy,1,5


### Answer the following questions: 
- What time of day were the most purchases made? We want to take a look at the average for the flights within each time period. 
- What seat position had the highest purchase amount? Is the aisle seat the highest earner because it's closest to the trolley?
- As Business Class purchases are free, how much is this costing us? 

In [430]:

df_econ = df_all.loc[df_all['Business Class'] == 'Economy']
df_econ

,first_name,last_name,passenger_number,flight_number,purchase_amount,Row,Seat Position,FlightID,DepAir,ArrAir,DepDate,DepTime,Depart Time of Day,FlightNo.,Business Class,Bclass_start,Bclass_end
30,Lacey,Joutapavicius,31,1,29.71,6,Window Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,Economy,1,5
31,Nicoline,Carneck,32,1,40.18,6,Middle Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,Economy,1,5
32,Loria,Ebbage,33,1,51.64,6,Aisle Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,Economy,1,5
33,Vanessa,Wheelan,34,1,0.00,6,Aisle Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,Economy,1,5
34,Harlie,Keatch,35,1,14.56,6,Middle Seats,1,LHR,SEA,2020-10-08,1900-01-01 14:53:00,Afternoon,1,Economy,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Skye,McLaverty,106,10,10.46,18,Aisle Seats,10,CAI,LHR,2020-12-22,1900-01-01 10:54:00,Morning,10,Economy,1,5
996,Margaux,Rymour,107,10,0.00,18,Middle Seats,10,CAI,LHR,2020-12-22,1900-01-01 10:54:00,Morning,10,Economy,1,5
997,Corny,Vaszoly,108,10,44.60,18,Window Seats,10,CAI,LHR,2020-12-22,1900-01-01 10:54:00,Morning,10,Economy,1,5
998,Vittorio,Rushbrook,109,10,0.00,19,Window Seats,10,CAI,LHR,2020-12-22,1900-01-01 10:54:00,Morning,10,Economy,1,5


In [431]:
# What time of day were the most purchases made? We want to take a look at the average for the flights within each time period. 

df_answer1=economy_df.groupby(['Depart Time of Day','flight_number'])['purchase_amount'].sum().reset_index()
df_answer1=df_answer1.groupby(['Depart Time of Day'])['purchase_amount'].mean().round(2).reset_index()
df_answer1['Rank']=df_answer1['purchase_amount'].rank(ascending=False).astype(int)

df_answer1.rename(columns={'purchase_amount': 'Avg per Flight'},inplace=True)

df_answer1 = df_answer1[['Rank','Depart Time of Day','Avg per Flight']] 
df_answer1

,Rank,Depart Time of Day,Avg per Flight
0,1,Morning,591.33
1,2,Afternoon,587.73
2,3,Evening,355.58


In [432]:
# What seat position had the highest purchase amount? Is the aisle seat the highest earner because it's closest to the trolley?
df_answer2 = df_econ.groupby(['Seat Position'])['purchase_amount'].sum().reset_index()
df_answer2['Rank']=df_answer2['purchase_amount'].rank(ascending=False).astype(int)

df_answer2 = df_answer2[['Rank','Seat Position','purchase_amount']].sort_values(by='Rank')
df_answer2

,Rank,Seat Position,purchase_amount
2,1,Window Seats,5660.42
0,2,Aisle Seats,4888.19
1,3,Middle Seats,4797.72


In [433]:
# As Business Class purchases are free, how much is this costing us? 

df_answer3 = df_all.groupby(['Business Class'])['purchase_amount'].sum().reset_index()
df_answer3['Rank']=df_answer3['purchase_amount'].rank(ascending=False).astype(int)


df_answer3 = df_answer3[['Rank','Business Class','purchase_amount']].sort_values(by='Rank')
df_answer3

,Rank,Business Class,purchase_amount
1,1,Economy,15346.33
0,2,Busniess,7207.69


### Combine all of the outputs into a single Excel workbook, with a different sheet for each output

In [434]:
# Create Excel writer object
writer = pd.ExcelWriter('output/2021-week14-output.xlsx')

# Write each DataFrame to a separate sheet in the Excel file
df_answer1.to_excel(writer, sheet_name='Time of Day', index=False)
df_answer2.to_excel(writer, sheet_name='Seat Position', index=False)
df_answer3.to_excel(writer, sheet_name='Business or Economy', index=False)